
# <span style="color:rgb(213,80,0)"> Control para Robot SCARA</span>

Documentacion:Torbellin Sanchez Mauricio Dariel


Fecha: 20\-10\-2025

## Descripción General

Este código implementa un nodo de ROS 2 para controlar un robot SCARA (Selective Compliance Articulated Robot Arm) que ejecuta una trayectoria lineal en el espacio cartesiano. El robot sigue una línea recta desde un punto inicial hasta un punto final durante un tiempo específico.

###  **Estructura del Código**
### **1.Shebang y Importaciones**

In [1]:
#!/usr/bin/env python3

import rclpy
from rclpy.node import Node
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from builtin_interfaces.msg import Duration

import time
from math import cos, sin, acos, asin, atan2, sqrt

### Propósito:
-  `#!/usr/bin/env python3`: Especifica el intérprete para ejecución directa
-  Mismas importaciones ROS 2 que el código anterior
-  Funciones matemáticas para cálculos de cinemática
###  **2. Clase Principal:**
### **`ScaraTrayLineNode`**Constructor `__init__`

In [2]:
def __init__(self):
    super().__init__("scara_tray_line_node")
    topic_name = "/scara_trajectory_controller/joint_trajectory"
    self.joints_ = ['link_1_joint', 'link_2_joint', 'link_3_joint']
    self.lamda_ = 0
    self.Tiempo_ejec_ = 10

### Variables de Instancia:
-  `joints_`: Nombres de las 3 articulaciones del SCARA
-  `lamda_`: Parámetro de interpolación (0 a 10)
-  `Tiempo_ejec_`: Tiempo total de ejecución en segundos
### Configuración del Publisher

In [3]:
    self.scara_tray_pub_ = self.create_publisher(
    JointTrajectory, topic_name, 10)


Topic: `/scara_trajectory_controller/joint_trajectory`

### Timer e Inicialización

In [4]:
    self.tray_timer_ = self.create_timer(1, self.trayectory_cbck)
self.get_logger().info('Scara activo, trayectoria linea recta')

###  **Función** **`trayectory_cbck`** **\- Control de Trayectoria** Estructura del Mensaje

In [5]:
trayectory_msg = JointTrajectory()
trayectory_msg.joint_names = self.joints_
point = JointTrajectoryPoint()

### Lógica de Control Principal

In [6]:
if self.lamda_ <= self.Tiempo_ejec_:
    # Definición de puntos inicial y final
    x_1 = 0.1
    y_1 = 0.6
    theta_1 = 0
    x_2 = 0.3
    y_2 = -0.6
    theta_2 = 1.57  # 90° en radianes

    # Cálculo de cinemática inversa
    solucion = invk_sol(self.lamda_, x_1, y_1, theta_1, x_2, y_2, theta_2)

    # Configuración del punto de trayectoria
    point.positions = solucion
    point.time_from_start = Duration(sec=1)
    trayectory_msg.points.append(point)

    # Publicación y logging
    self.scara_tray_pub_.publish(trayectory_msg)
    self.get_logger().info("Postura actual {}".format(solucion))
    time.sleep(2)
    self.lamda_ += 1

### Puntos de Trayectoria:
-  Inicio: (x=0.1, y=0.6, θ=0 rad)
-  Fin: (x=0.3, y=\-0.6, θ=1.57 rad ≈ 90°)
### Condición de Finalización

In [7]:
    elif self.lamda_ > 10:
    link_1_joint = 0
    link_2_joint = 0
    link_3_joint = 0
    return [float(link_1_joint), float(link_2_joint), float(link_3_joint)]


Nota: Esta condición parece tener un error, ya que el `return` no afecta el flujo principal.


 **Función** **`invk_sol`** **\- Cinemática Inversa del SCARA** Parámetros del Robot


In [8]:
    Tiempo_ejec_ = 10
L_1 = 0.5  # Longitud del primer eslabón
L_2 = 0.5  # Longitud del segundo eslabón
L_3 = 0.3  # Longitud del tercer eslabón (prismático o rotacional)

### Interpolación Lineal Cartesiana

In [9]:
x_P = x_in + (param/Tiempo_ejec_)*(x_fin - x_in)
y_P = y_in + (param/Tiempo_ejec_)*(y_fin - y_in)
theta_P = theta_in + (param/Tiempo_ejec_)*(theta_fin - theta_in)

### Fórmula de Interpolación:

In [10]:
valor_actual = valor_inicial + (t/T) * (valor_final - valor_inicial)


Donde `t = param` y `T = Tiempo_ejec_`Cálculo de Posición del Efector Final


In [11]:
x_3 = x_P - L_3*cos(theta_P)
y_3 = y_P - L_3*sin(theta_P)


Propósito: Calcula la posición de la articulación 3 basado en la posición y orientación del efector final.

### Resolución de Cinemática Inversa

In [12]:
theta_2 = acos((pow(x_3, 2)+pow(y_3,2)-pow(L_1, 2)-pow(L_2, 2))/(2*L_1*L_2))
beta = atan2(y_3, x_3)
psi = acos((pow(x_3, 2)+pow(y_3,2)+pow(L_1, 2)-pow(L_2, 2))/(2*L_1*sqrt(pow(x_3, 2)+pow(y_3,2))))
theta_1 = beta - psi
theta_3 = theta_P - theta_1 - theta_2

### Explicación de Ángulos:
-  `theta_2`: Ángulo de la segunda articulación (ley del coseno)
-  `beta`: Ángulo hacia el punto (x₃, y₃) desde el origen
-  `psi`: Ángulo auxiliar para cálculo de theta\_1
-  `theta_1`: Ángulo de la primera articulación
-  `theta_3`: Ángulo de la tercera articulación (orientación final)
### **5. Función Principal**

In [13]:
def main(args=None):
    rclpy.init(args=args)
    node = ScaraTrayLineNode()
    rclpy.spin(node)
    rclpy.shutdown()

### Características del Robot SCARA:
-  Grados de libertad: 3 (2 rotacionales + 1 rotacional/prismático)
-  Longitudes de eslabones: L₁=0.5m, L₂=0.5m, L₃=0.3m
-  Tipo de trayectoria: Lineal en espacio cartesiano
### **Parámetros de Trayectoria:**
-  Tiempo total: 10 segundos
-  Frecuencia de actualización: 1 Hz (timer de 1 segundo)
-  Delay entre puntos: 2 segundos
-  Total de puntos: 11 (de λ=0 a λ=10)
### **Topics y Mensajes:**
-  Topic: `/scara_trajectory_controller/joint_trajectory`
-  Tipo de mensaje: `JointTrajectory`
-  Joints: `['link_1_joint', 'link_2_joint', 'link_3_joint']`
## Análisis de la Trayectoria

Movimiento Cartesiano:

-  Desplazamiento X: 0.1m → 0.3m (+0.2m)
-  Desplazamiento Y: 0.6m → \-0.6m (\-1.2m)
-  Rotación: 0° → 90° (1.57 rad)
### **Patrón de Ejecución:**
1.  Inicialización: λ = 0, posición inicial
2. Iteraciones: λ = 1 a 10, interpolación lineal
3. Finalización: λ > 10, se detiene (pero no retorna a cero correctamente)
